In [ ]:
!pip install openai-whisper==20230117

In [2]:
! pip install gradio -q

In [3]:
import gradio as gr
import os
import sys
import subprocess

c:\Users\KRISH\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import whisper

model = whisper.load_model("medium")

c:\Users\KRISH\AppData\Local\Programs\Python\Python312\Lib\site-packages\whisper\__init__.py:110: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, m

In [5]:
def video2mp3(video_file, output_ext="mp3"):
    filename, ext = os.path.splitext(video_file)
    subprocess.call(["ffmpeg", "-y", "-i", video_file, f"{filename}.{output_ext}"],
                    stdout=subprocess.DEVNULL,
                    stderr=subprocess.STDOUT)
    return f"{filename}.{output_ext}"


In [ ]:
#input_video = 'this+invention+makes+getting+out+of+bed+so+much+easier..mp4'

input_video = '/content/tyty.mp4'

In [ ]:
audio_file = video2mp3(input_video)

In [ ]:
from IPython.display import Audio
Audio(audio_file)

In [ ]:
def translate(audio):

    options = dict(beam_size=5, best_of=5)
    translate_options = dict(task="translate", **options)
    result = model.transcribe(audio_file,**translate_options)
    return result

In [ ]:
result = translate(audio_file)

In [ ]:
print(result["text"])

In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open(input_video,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
from whisper.utils import write_vtt

In [ ]:
output_dir = '/content/'
audio_path = audio_file.split(".")[0]

In [ ]:
with open(os.path.join(output_dir, audio_path + ".vtt"), "w") as vtt:
    write_vtt(result["segments"], file=vtt)

In [ ]:
subtitle = audio_path + ".vtt"
output_video = audio_path + "_subtitled.mp4"

In [ ]:
os.system(f"ffmpeg -i {input_video} -vf subtitles={subtitle} {output_video}")

In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open(output_video,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
import gradio as gr
import os
import sys
import subprocess

import whisper
from whisper.utils import write_vtt

model = whisper.load_model("medium")

def video2mp3(video_file, output_ext="mp3"):
    filename, ext = os.path.splitext(video_file)
    subprocess.call(["ffmpeg", "-y", "-i", video_file, f"{filename}.{output_ext}"],
                    stdout=subprocess.DEVNULL,
                    stderr=subprocess.STDOUT)
    return f"{filename}.{output_ext}"


def translate(input_video):

    audio_file = video2mp3(input_video)

    options = dict(beam_size=5, best_of=5)
    translate_options = dict(task="translate", **options)
    result = model.transcribe(audio_file,**translate_options)

    output_dir = '/content/'
    audio_path = audio_file.split(".")[0]

    with open(os.path.join(output_dir, audio_path + ".vtt"), "w") as vtt:
      write_vtt(result["segments"], file=vtt)

    subtitle = audio_path + ".vtt"
    output_video = audio_path + "_subtitled.mp4"

    os.system(f"ffmpeg -i {input_video} -vf subtitles={subtitle} {output_video}")

    return output_video



title = "Add Text/Caption to your YouTube Shorts - MultiLingual"

block = gr.Blocks()

with block:
    with gr.Group():
        with gr.Row():
            inp_video = gr.Video(
                label="Input Video"
            )
            op_video = gr.Video()
        with gr.Row():
            btn = gr.Button("Generate Subtitle Video")
            btn.click(translate, inputs=[inp_video], outputs=[op_video])

        gr.HTML('''
        <div class="footer">
                    <p>Model by <a href="https://github.com/openai/whisper" style="text-decoration: underline;" target="_blank">OpenAI</a> - Gradio App by <a href="https://twitter.com/1littlecoder" style="text-decoration: underline;" target="_blank">1littlecoder</a>
                    </p>
        </div>
        ''')

block.launch(debug = True)